# Import libraries

In [1]:
import torch
import json
from torch import nn
from tool import utils, preprocess, architecture, analysis
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import time
import os

# Before the analysis ...

## Get user input

In [2]:
ANALYSIS_NAME = 'Model1_Preproc' # Specify the folder name here (ex ANALYSIS_NAME = 'Model1_Preproc') ...
ANALYSIS_DIR_PATH = os.path.join('./RUN', ANALYSIS_NAME)
params = analysis.get_params(ANALYSIS_DIR_PATH)
params

{'ARCHITECTURE': '1', 'TREATMENT': 'yes'}

## Build test data loader

In [3]:
test_loader, img_size = analysis.get_test_loader(params)

Compose(
    ToTensor()
    Normalize(mean=[0.1292, 0.0888, 0.0874], std=[0.4607, 0.4705, 0.445])
)


/Users/davidlin/.pyenv/versions/general-virtual-env-3.7.9/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## Build and load CNN model

In [4]:
model = analysis.build_load_model(params, ANALYSIS_DIR_PATH, img_size)

In [5]:
model

CNN_Model1(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1228800, out_features=18, bias=True)
)

# Analyses starts here ...

## Make predictions 

In [9]:
since = time.time()

true_label, predict = analysis.predict(model, test_loader)

time_elapsed = time.time() - since
print('Predictions on {} test images takes {:.0f}m {:.0f}s'.format(
    len(test_loader.dataset), time_elapsed // 60, time_elapsed % 60))

cool
cool
cool


FileNotFoundError: [Errno 2] No such file or directory: './Trail_dataset/test_data/R_2/-0.1331631_2002.jpeg'

In [8]:
true_label

NameError: name 'true_label' is not defined

## Accuracy

In [7]:
np.mean(true_label == predict) # 0.87

NameError: name 'true_label' is not defined

## Confusion matrix

In [ ]:
con = confusion_matrix(true_label, predict)
con_df = pd.DataFrame(con)
con_df.columns = test_loader.dataset.classes
con_df.index = test_loader.dataset.classes

In [ ]:
con_df

## Training logs

In [ ]:
history = pd.read_csv(os.path.join(ANALYSIS_DIR_PATH, 'training_logger.csv'))

In [ ]:
history